In [1]:
import osmnx
import pandas as pd
import geopandas as gpd

This notebook loads an osmnx network graph and maps the stops of a trajectory file to the nodes of the network.

In [2]:
if "snakemake" in locals():
    trajectories_path = snakemake.input["trajectories"]
    network_path = snakemake.input["network"]
    output_path = snakemake.output[0]
else:
    trajectories_path = "../../resources/operators/synthetic1/trajectories.csv"
    network_path = "../../results/network/network.graphml"
    output_path = "../../results/testing/stops.parquet"

In [3]:
# Load graph
osmnx.io._convert_bool_string = lambda x: bool(x) if not x is None else False
graph = osmnx.load_graphml(network_path)

In [4]:
# Process trajectories
df_trajectories = pd.read_csv(trajectories_path)[["longitude", "latitude"]]
df_trajectories["geometry"] = gpd.points_from_xy(df_trajectories["longitude"], df_trajectories["latitude"])
df_trajectories = gpd.GeoDataFrame(df_trajectories, crs = "EPSG:4326")

In [5]:
# Find the closest node
df_trajectories["node"] = osmnx.nearest_nodes(graph, 
    df_trajectories["geometry"].x, df_trajectories["geometry"].y)

In [8]:
# Write nodes
df_trajectories[["node"]].to_parquet(output_path)